# Movie Recommender System (Content Based)

In [10]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import joblib

In [11]:
url1 = "Datasets/movies.csv"
url2 = "Datasets/credits.csv"

df1 = pd.read_csv(url1)
df2 = pd.read_csv(url2)
df1.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [12]:
df = df1.merge(df2, on="title")
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [13]:
df.isnull().sum()

budget                     0
genres                     0
homepage                3096
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
movie_id                   0
cast                       0
crew                       0
dtype: int64

In [14]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [15]:
df.drop(['budget', 'homepage', 'original_language',
       'original_title', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'vote_average',
       'vote_count'],
axis=1, inplace=True)
df.head()

,genres,id,keywords,overview,title,movie_id,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,Spectre,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,The Dark Knight Rises,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",John Carter,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [16]:
df.isnull().sum()

genres      0
id          0
keywords    0
overview    3
title       0
movie_id    0
cast        0
crew        0
dtype: int64

In [17]:
df.dropna(inplace=True)
df.isnull().sum()

genres      0
id          0
keywords    0
overview    0
title       0
movie_id    0
cast        0
crew        0
dtype: int64

#### Making Genres and Keywords as List

In [18]:
df["genres"][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [19]:
def convert(objects):
    object_list = []
    for object in ast.literal_eval(objects):
        object_list.append(object["name"])
    return object_list

In [20]:
convert(df["genres"][0])

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [21]:
df["genres"].apply(convert)

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4804                        [Action, Crime, Thriller]
4805                                [Comedy, Romance]
4806               [Comedy, Drama, Romance, TV Movie]
4807                                               []
4808                                    [Documentary]
Name: genres, Length: 4806, dtype: object

In [22]:
df["genres"] = df["genres"].apply(convert)
df["keywords"] = df["keywords"].apply(convert)

In [23]:
df.head()

,genres,id,keywords,overview,title,movie_id,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Avatar,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Spectre,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,The Dark Knight Rises,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",John Carter,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


#### Making Actors List

In [24]:
def fetch_actor(objects):
    object_list = []
    counter = 0
    for object in ast.literal_eval(objects):
        if counter != 3:
            object_list.append(object["name"])
            counter += 1
        else: 
            break
    print(object_list)
    return object_list

In [25]:
fetch_actor(df["cast"][0])

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']


['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']

In [26]:
df["cast"] = df["cast"].apply(fetch_actor)
df.head()

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']
['Johnny Depp', 'Orlando Bloom', 'Keira Knightley']
['Daniel Craig', 'Christoph Waltz', 'Léa Seydoux']
['Christian Bale', 'Michael Caine', 'Gary Oldman']
['Taylor Kitsch', 'Lynn Collins', 'Samantha Morton']
['Tobey Maguire', 'Kirsten Dunst', 'James Franco']
['Zachary Levi', 'Mandy Moore', 'Donna Murphy']
['Robert Downey Jr.', 'Chris Hemsworth', 'Mark Ruffalo']
['Daniel Radcliffe', 'Rupert Grint', 'Emma Watson']
['Ben Affleck', 'Henry Cavill', 'Gal Gadot']
['Brandon Routh', 'Kevin Spacey', 'Kate Bosworth']
['Daniel Craig', 'Olga Kurylenko', 'Mathieu Amalric']
['Johnny Depp', 'Orlando Bloom', 'Keira Knightley']
['Johnny Depp', 'Armie Hammer', 'William Fichtner']
['Henry Cavill', 'Amy Adams', 'Michael Shannon']
['Ben Barnes', 'William Moseley', 'Anna Popplewell']
['Robert Downey Jr.', 'Chris Evans', 'Mark Ruffalo']
['Johnny Depp', 'Penélope Cruz', 'Ian McShane']
['Will Smith', 'Tommy Lee Jones', 'Josh Brolin']
['Martin Freeman', 'Ian 

,genres,id,keywords,overview,title,movie_id,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Avatar,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,285,"[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Spectre,206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux]","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,The Dark Knight Rises,49026,"[Christian Bale, Michael Caine, Gary Oldman]","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",John Carter,49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton]","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


#### Making Director List

In [27]:
def fetch_director(objects):
    object_list = []
    for object in ast.literal_eval(objects):
        if object["job"] == "Director":
            object_list.append(object["name"])
    
    print(object_list)  
    return object_list

In [28]:
fetch_director(df["crew"][0])

['James Cameron']


['James Cameron']

In [29]:
df["crew"] = df["crew"].apply(fetch_director)
df.head()

['James Cameron']
['Gore Verbinski']
['Sam Mendes']
['Christopher Nolan']
['Andrew Stanton']
['Sam Raimi']
['Byron Howard', 'Nathan Greno']
['Joss Whedon']
['David Yates']
['Zack Snyder']
['Bryan Singer']
['Marc Forster']
['Gore Verbinski']
['Gore Verbinski']
['Zack Snyder']
['Andrew Adamson']
['Joss Whedon']
['Rob Marshall']
['Barry Sonnenfeld']
['Peter Jackson']
['Marc Webb']
['Ridley Scott']
['Peter Jackson']
['Chris Weitz']
['Peter Jackson']
['James Cameron']
['Anthony Russo', 'Joe Russo']
['Peter Berg']
['Colin Trevorrow']
['Sam Mendes']
['Sam Raimi']
['Shane Black']
['Tim Burton']
['Brett Ratner']
['Dan Scanlon']
['Michael Bay']
['Michael Bay']
['Sam Raimi']
['Marc Webb']
['Joseph Kosinski']
['John Lasseter', 'Brad Lewis']
['Martin Campbell']
['Lee Unkrich']
['McG']
['James Wan']
['Marc Forster']
['Bryan Singer']
['J.J. Abrams']
['Bryan Singer']
['Baz Luhrmann']
['Mike Newell']
['Guillermo del Toro']
['Michael Bay']
['Steven Spielberg']
['Peter Sohn']
['Brenda Chapman', 'Mark And

,genres,id,keywords,overview,title,movie_id,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Avatar,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,285,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Spectre,206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,The Dark Knight Rises,49026,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",John Carter,49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


#### Making Overview as List

In [30]:
df["overview"] = df["overview"].apply(lambda x: x.split())
df.head()

,genres,id,keywords,overview,title,movie_id,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...","[In, the, 22nd, century,, a, paraplegic, Marin...",Avatar,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...","[Captain, Barbossa,, long, believed, to, be, d...",Pirates of the Caribbean: At World's End,285,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...","[A, cryptic, message, from, Bond’s, past, send...",Spectre,206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...","[Following, the, death, of, District, Attorney...",The Dark Knight Rises,49026,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...","[John, Carter, is, a, war-weary,, former, mili...",John Carter,49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


#### Removing Spaces

In [31]:
df["genres"].apply(lambda x: [i.replace(" ", "") for i in x])

0       [Action, Adventure, Fantasy, ScienceFiction]
1                       [Adventure, Fantasy, Action]
2                         [Action, Adventure, Crime]
3                   [Action, Crime, Drama, Thriller]
4                [Action, Adventure, ScienceFiction]
                            ...                     
4804                       [Action, Crime, Thriller]
4805                               [Comedy, Romance]
4806               [Comedy, Drama, Romance, TVMovie]
4807                                              []
4808                                   [Documentary]
Name: genres, Length: 4806, dtype: object

In [32]:
df.columns

Index(['genres', 'id', 'keywords', 'overview', 'title', 'movie_id', 'cast',
       'crew'],
      dtype='object')

In [33]:
df["genres"] = df["genres"].apply(lambda x: [i.replace(" ", "") for i in x])
df["keywords"] = df["keywords"].apply(lambda x: [i.replace(" ", "") for i in x])
df["overview"] = df["overview"].apply(lambda x: [i.replace(" ", "") for i in x])
df["cast"] = df["cast"].apply(lambda x: [i.replace(" ", "") for i in x])
df["crew"] = df["crew"].apply(lambda x: [i.replace(" ", "") for i in x])

In [34]:
df.head()

,genres,id,keywords,overview,title,movie_id,cast,crew
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...",Avatar,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...",Pirates of the Caribbean: At World's End,285,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,"[Action, Adventure, Crime]",206647,"[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send...",Spectre,206647,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,"[Action, Crime, Drama, Thriller]",49026,"[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney...",The Dark Knight Rises,49026,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,"[Action, Adventure, ScienceFiction]",49529,"[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili...",John Carter,49529,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


## Step 1: Making Tags

In [35]:
df["tags"] = df["genres"] + df["keywords"] + df["overview"] + df["cast"] + df["crew"]

In [36]:
df["tags"][0]

['Action',
 'Adventure',
 'Fantasy',
 'ScienceFiction',
 'cultureclash',
 'future',
 'spacewar',
 'spacecolony',
 'society',
 'spacetravel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alienplanet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'loveaffair',
 'antiwar',
 'powerrelations',
 'mindandsoul',
 '3d',
 'In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.',
 'SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron']

In [37]:
df.head()

,genres,id,keywords,overview,title,movie_id,cast,crew,tags
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...",Avatar,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction, c..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...",Pirates of the Caribbean: At World's End,285,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Adventure, Fantasy, Action, ocean, drugabuse,..."
2,"[Action, Adventure, Crime]",206647,"[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send...",Spectre,206647,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[Action, Adventure, Crime, spy, basedonnovel, ..."
3,"[Action, Crime, Drama, Thriller]",49026,"[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney...",The Dark Knight Rises,49026,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Action, Crime, Drama, Thriller, dccomics, cri..."
4,"[Action, Adventure, ScienceFiction]",49529,"[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili...",John Carter,49529,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[Action, Adventure, ScienceFiction, basedonnov..."


In [38]:
new_df = df[["movie_id", "title", "tags"]]
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, c..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, ocean, drugabuse,..."
2,206647,Spectre,"[Action, Adventure, Crime, spy, basedonnovel, ..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, dccomics, cri..."
4,49529,John Carter,"[Action, Adventure, ScienceFiction, basedonnov..."


In [39]:
new_df["tags"][0]

['Action',
 'Adventure',
 'Fantasy',
 'ScienceFiction',
 'cultureclash',
 'future',
 'spacewar',
 'spacecolony',
 'society',
 'spacetravel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alienplanet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'loveaffair',
 'antiwar',
 'powerrelations',
 'mindandsoul',
 '3d',
 'In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.',
 'SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron']

In [40]:
new_df["tags"].apply(lambda x: " ".join(x))

0       Action Adventure Fantasy ScienceFiction cultur...
1       Adventure Fantasy Action ocean drugabuse exoti...
2       Action Adventure Crime spy basedonnovel secret...
3       Action Crime Drama Thriller dccomics crimefigh...
4       Action Adventure ScienceFiction basedonnovel m...
                              ...                        
4804    Action Crime Thriller unitedstates–mexicobarri...
4805    Comedy Romance A newlywed couple's honeymoon i...
4806    Comedy Drama Romance TVMovie date loveatfirsts...
4807    When ambitious New York attorney Sam is sent t...
4808    Documentary obsession camcorder crush dreamgir...
Name: tags, Length: 4806, dtype: object

In [41]:
new_df["tags"] = new_df["tags"].apply(lambda x: " ".join(x))

C:\Users\sadma\AppData\Local\Temp\ipykernel_2972\3719869087.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"] = new_df["tags"].apply(lambda x: " ".join(x))


In [42]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,Action Adventure Fantasy ScienceFiction cultur...
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action ocean drugabuse exoti...
2,206647,Spectre,Action Adventure Crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,Action Crime Drama Thriller dccomics crimefigh...
4,49529,John Carter,Action Adventure ScienceFiction basedonnovel m...


In [43]:
new_df["tags"][0]

'Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [44]:
new_df["tags"] = new_df["tags"].apply(lambda x: x.lower())

C:\Users\sadma\AppData\Local\Temp\ipykernel_2972\1654334444.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"] = new_df["tags"].apply(lambda x: x.lower())


In [45]:
new_df["tags"][0]

'action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. samworthington zoesaldana sigourneyweaver jamescameron'

In [46]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,action adventure fantasy sciencefiction cultur...
1,285,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drugabuse exoti...
2,206647,Spectre,action adventure crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,action crime drama thriller dccomics crimefigh...
4,49529,John Carter,action adventure sciencefiction basedonnovel m...


## Step 2: Making Vector

In [47]:
cv = CountVectorizer(max_features=5000, stop_words="english")
vectors = cv.fit_transform(new_df["tags"]).toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4806, 5000))

In [48]:
features_names = list(cv.get_feature_names_out())
print(features_names[-10:])

['zhangziyi', 'zion', 'zoe', 'zoesaldana', 'zombie', 'zombieapocalypse', 'zombies', 'zone', 'zoo', 'zooeydeschanel']


## Step 3: Steaming - removing duplicate words like running, runner to run

In [49]:
ps = PorterStemmer()
ps.stem("playing")

'play'

In [50]:
def stem(text):
    words_list = []
    for word in text.split():
        item = ps.stem(word)
        words_list.append(item)
    return " ".join(words_list)

In [51]:
new_df["tags"].apply(stem)

0       action adventur fantasi sciencefict culturecla...
1       adventur fantasi action ocean drugabus exotici...
2       action adventur crime spi basedonnovel secreta...
3       action crime drama thriller dccomic crimefight...
4       action adventur sciencefict basedonnovel mar m...
                              ...                        
4804    action crime thriller unitedstates–mexicobarri...
4805    comedi romanc a newlyw couple' honeymoon is up...
4806    comedi drama romanc tvmovi date loveatfirstsig...
4807    when ambiti new york attorney sam is sent to s...
4808    documentari obsess camcord crush dreamgirl eve...
Name: tags, Length: 4806, dtype: object

In [52]:
new_df["tags"] = new_df["tags"].apply(stem)

C:\Users\sadma\AppData\Local\Temp\ipykernel_2972\825579503.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"] = new_df["tags"].apply(stem)


In [53]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,action adventur fantasi sciencefict culturecla...
1,285,Pirates of the Caribbean: At World's End,adventur fantasi action ocean drugabus exotici...
2,206647,Spectre,action adventur crime spi basedonnovel secreta...
3,49026,The Dark Knight Rises,action crime drama thriller dccomic crimefight...
4,49529,John Carter,action adventur sciencefict basedonnovel mar m...


## Step 4: Compute Similarity

In [54]:
cosine_similarity(vectors)

array([[1.        , 0.08858079, 0.05905386, ..., 0.02478408, 0.02599376,
        0.        ],
       [0.08858079, 1.        , 0.06451613, ..., 0.02707652, 0.        ,
        0.        ],
       [0.05905386, 0.06451613, 1.        , ..., 0.02707652, 0.        ,
        0.        ],
       ...,
       [0.02478408, 0.02707652, 0.02707652, ..., 1.        , 0.07150969,
        0.0489116 ],
       [0.02599376, 0.        , 0.        , ..., 0.07150969, 1.        ,
        0.05129892],
       [0.        , 0.        , 0.        , ..., 0.0489116 , 0.05129892,
        1.        ]], shape=(4806, 4806))

In [55]:
similarity = cosine_similarity(vectors)
similarity[0]

array([1.        , 0.08858079, 0.05905386, ..., 0.02478408, 0.02599376,
       0.        ], shape=(4806,))

##### Finding Movie index

In [56]:
new_df["title"] == "Avatar"

0        True
1       False
2       False
3       False
4       False
        ...  
4804    False
4805    False
4806    False
4807    False
4808    False
Name: title, Length: 4806, dtype: bool

In [57]:
new_df[new_df["title"] == "Avatar"]

,movie_id,title,tags
0,19995,Avatar,action adventur fantasi sciencefict culturecla...


In [58]:
new_df[new_df["title"] == "Avatar"].index[0]

np.int64(0)

In [59]:
# movie index
new_df[new_df["title"] == "Batman Begins"].index[0]

np.int64(119)

#### Distance

In [60]:
similarity[0]

array([1.        , 0.08858079, 0.05905386, ..., 0.02478408, 0.02599376,
       0.        ], shape=(4806,))

In [61]:
enumerate(similarity[0])

In [62]:
list(enumerate(similarity[0]))

[(0, np.float64(0.9999999999999998)),
 (1, np.float64(0.0885807893036239)),
 (2, np.float64(0.059053859535749265)),
 (3, np.float64(0.03874921291460643)),
 (4, np.float64(0.17757120130114434)),
 (5, np.float64(0.11510231387903269)),
 (6, np.float64(0.01964943729729648)),
 (7, np.float64(0.17155007848855516)),
 (8, np.float64(0.06213697660012)),
 (9, np.float64(0.07435223956449145)),
 (10, np.float64(0.1144727614064285)),
 (11, np.float64(0.07897471897389846)),
 (12, np.float64(0.0949157995752499)),
 (13, np.float64(0.04649905549752771)),
 (14, np.float64(0.11115400287534567)),
 (15, np.float64(0.04559607525875532)),
 (16, np.float64(0.08000711205939975)),
 (17, np.float64(0.1461508458653816)),
 (18, np.float64(0.1098436937909367)),
 (19, np.float64(0.08419653359482289)),
 (20, np.float64(0.056573571480353566)),
 (21, np.float64(0.09158437663089952)),
 (22, np.float64(0.06711560552140244)),
 (23, np.float64(0.0949157995752499)),
 (24, np.float64(0.05479966243511909)),
 (25, np.float64(0

In [63]:
sorted(list(enumerate(similarity[0])), reverse=True)

[(4805, np.float64(0.0)),
 (4804, np.float64(0.025993762245501817)),
 (4803, np.float64(0.024784079854830485)),
 (4802, np.float64(0.051987524491003634)),
 (4801, np.float64(0.024239308278185982)),
 (4800, np.float64(0.0)),
 (4799, np.float64(0.056388387443460086)),
 (4798, np.float64(0.020712325533373332)),
 (4797, np.float64(0.0)),
 (4796, np.float64(0.0)),
 (4795, np.float64(0.0)),
 (4794, np.float64(0.0)),
 (4793, np.float64(0.058123819371909635)),
 (4792, np.float64(0.0)),
 (4791, np.float64(0.0)),
 (4790, np.float64(0.03106848830006)),
 (4789, np.float64(0.022371868507134143)),
 (4788, np.float64(0.0)),
 (4787, np.float64(0.02177517781546711)),
 (4786, np.float64(0.0)),
 (4785, np.float64(0.0)),
 (4784, np.float64(0.04393747751637468)),
 (4783, np.float64(0.0)),
 (4782, np.float64(0.02666903735313325)),
 (4781, np.float64(0.06213697660012)),
 (4780, np.float64(0.022167554926073633)),
 (4779, np.float64(0.0)),
 (4778, np.float64(0.0)),
 (4777, np.float64(0.051987524491003634)),
 (

In [64]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x: x[1])

[(0, np.float64(0.9999999999999998)),
 (539, np.float64(0.2537477434955704)),
 (1192, np.float64(0.25112360116696136)),
 (260, np.float64(0.2442250043490654)),
 (1214, np.float64(0.24260844762468367)),
 (507, np.float64(0.24165612634006073)),
 (582, np.float64(0.2372894989381248)),
 (1440, np.float64(0.2372894989381248)),
 (1202, np.float64(0.23692415692169538)),
 (1916, np.float64(0.23394386020951635)),
 (3728, np.float64(0.2324952774876386)),
 (74, np.float64(0.22305671869347438)),
 (322, np.float64(0.21919864974047634)),
 (3606, np.float64(0.21775177815467112)),
 (83, np.float64(0.21582051197002577)),
 (4190, np.float64(0.2152488010002526)),
 (2329, np.float64(0.2113701265354594)),
 (2405, np.float64(0.21059925059706258)),
 (47, np.float64(0.21049133398705727)),
 (61, np.float64(0.21029989116983447)),
 (495, np.float64(0.21029989116983447)),
 (972, np.float64(0.2101050788156825)),
 (2969, np.float64(0.2071841649822589)),
 (942, np.float64(0.20668850837512243)),
 (1327, np.float64(0.

In [65]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x: x[1])[1:6]

[(539, np.float64(0.2537477434955704)),
 (1192, np.float64(0.25112360116696136)),
 (260, np.float64(0.2442250043490654)),
 (1214, np.float64(0.24260844762468367)),
 (507, np.float64(0.24165612634006073))]

## Step 5: Recommand

In [66]:
def recom(movie):
    movie_index = new_df[new_df["title"] == movie].index[0]
    distance = similarity[movie_index]
    movie_list = sorted(list(enumerate(distance)), reverse=True, key=lambda x: x[1])[1:6]
    
    for movie in movie_list:
        print(movie[0])

In [67]:
recom("Avatar")

539
1192
260
1214
507


In [68]:
new_df.iloc[539]

movie_id                                                 7450
title                                              Titan A.E.
tags        anim action sciencefict famili adventur monste...
Name: 539, dtype: object

In [69]:
new_df.iloc[539].title

'Titan A.E.'

In [70]:
def sugg(movie):
    movie_index = new_df[new_df["title"] == movie].index[0]
    distance = similarity[movie_index]
    movie_list = sorted(list(enumerate(distance)), reverse=True, key=lambda x: x[1])[1:6]
    
    for movie in movie_list:
        print(new_df.iloc[movie[0]].title)

In [71]:
sugg("Batman Begins")

The Dark Knight
The Dark Knight Rises
Batman
Batman
Batman & Robin


In [72]:
def recommend(movie):
    movie_index = new_df[new_df["title"] == movie].index[0]
    distance = similarity[movie_index]
    movie_list = sorted(list(enumerate(distance)), reverse=True, key=lambda x: x[1])[1:6]
    
    recommendations = []
    for movie in movie_list:
        recommendations.append(new_df.iloc[movie[0]].title)
    return recommendations

In [73]:
recommend("Avatar")

['Titan A.E.',
 'Small Soldiers',
 "Ender's Game",
 'Aliens vs Predator: Requiem',
 'Independence Day']

In [74]:
model_data = {
    "data": new_df,
    "similarity": similarity
}

In [75]:
joblib.dump(model_data, "model.pkl")
print("Model Saved Successfully!")

Model Saved Successfully!


In [78]:
loaded_model = joblib.load("model.pkl")
loaded_model_prediction = recommend("Batman Begins")
print(f"Prediction from loaded model: {loaded_model_prediction}")

for movie in loaded_model_prediction:
    print(movie)

Prediction from loaded model: ['The Dark Knight', 'The Dark Knight Rises', 'Batman', 'Batman', 'Batman & Robin']
The Dark Knight
The Dark Knight Rises
Batman
Batman
Batman & Robin
